<a href="https://colab.research.google.com/github/villasen/colab_notebooks/blob/master/dscnn_UrbanSounds_172.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TRAINING AND TESTING URBAN SOUNDS

In [12]:
#%env URBAN_WORDS=Helicopter,Insects,Train,Dog,Snoring,Wind,Cat,Airplane,Breathing,Clapping
%env URBAN_WORDS=clock_alarm,sneezing,crying_baby,door_knock,drinking_sipping,dog,coughing,vacuum_cleaner,glass_breaking,laughing
#%env URBAN_WORDS=clock_alarm,laughing
%env TIME_CLIP=1000
%env SAMPLE_RATE=8000
%env TRAIN_STEPS=5000,3000,2000
%env MODEL_SIZE=5 172 10 4 2 1 172 3 3 2 2 172 3 3 1 1 172 3 3 1 1 172 3 3 1 1

env: URBAN_WORDS=clock_alarm,sneezing,crying_baby,door_knock,drinking_sipping,dog,coughing,vacuum_cleaner,glass_breaking,laughing
env: TIME_CLIP=1000
env: SAMPLE_RATE=8000
env: TRAIN_STEPS=5000,3000,2000
env: MODEL_SIZE=5 172 10 4 2 1 172 3 3 2 2 172 3 3 1 1 172 3 3 1 1 172 3 3 1 1


In [0]:
!rm -r work

In [0]:
!ls -l work/DS_CNN_UrbanSounds/DS_CNN1/training/best

In [0]:
!rm -r ML-KWS-for-MCU

In [0]:
!git clone https://github.com/villasen/ML-KWS-for-MCU.git


In [2]:
!git clone https://github.com/villasen/Sound-Datasets.git

Cloning into 'Sound-Datasets'...
remote: Enumerating objects: 1906, done.
remote: Counting objects: 100% (1906/1906), done.
remote: Compressing objects: 100% (1902/1902), done.
remote: Total 20691 (delta 8), reused 1897 (delta 4), pack-reused 18785
Receiving objects: 100% (20691/20691), 1.45 GiB | 34.55 MiB/s, done.
Resolving deltas: 100% (5535/5535), done.
Checking out files: 100% (14765/14765), done.


In [0]:
!rm -r Sound-Datasets

Train model for Speech Commands

In [10]:
!python ML-KWS-for-MCU/train.py \
--data_url= none  \
--data_dir Sound-Datasets/ESC50-downsampled_silence_dataset/ \
--summaries_dir work/DS_CNN_UrbanSounds/DS_CNN1/retrain_logs \
--train_dir work/DS_CNN_UrbanSounds/DS_CNN1/training \
--clip_duration_ms $TIME_CLIP  \
--window_size_ms 40.0 \
--window_stride_ms 20.0 \
--dct_coefficient_count 10 \
--testing_percentage 10 \
--sample_rate $SAMPLE_RATE \
--batch_size 100 \
--save_step_interval 200 \
--eval_step_interval 400 \
--validation_percentage 10 \
--unknown_percentage 10.0 \
--silence_percentage 10.0 \
--how_many_training_steps $TRAIN_STEPS  \
--model_size_info $MODEL_SIZE \
--wanted_words=$URBAN_WORDS

2019-04-20 02:01:38.444985: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-04-20 02:01:38.445290: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x5632510b2940 executing computations on platform Host. Devices:
2019-04-20 02:01:38.445348: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-04-20 02:01:38.632310: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-04-20 02:01:38.632900: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x5632510b3700 executing computations on platform CUDA. Devices:
2019-04-20 02:01:38.632978: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2019-04-20 02:01:38.633423: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1433

In [13]:
!python ML-KWS-for-MCU/test.py \
--data_url= none  \
--data_dir Sound-Datasets/ESC50-downsampled_silence_dataset/ \
--summaries_dir work/DS_CNN_UrbanSounds/DS_CNN1/retrain_logs \
--clip_duration_ms $TIME_CLIP  \
--window_size_ms 40.0 \
--window_stride_ms 20.0 \
--dct_coefficient_count 10 \
--testing_percentage 10 \
--sample_rate $SAMPLE_RATE \
--batch_size 100 \
--validation_percentage 10 \
--unknown_percentage 10.0 \
--silence_percentage 10.0 \
--how_many_training_steps $TRAIN_STEPS  \
--model_size_info $MODEL_SIZE \
--checkpoint work/DS_CNN_UrbanSounds/DS_CNN1/training/best/ds_cnn_6444.ckpt-3600 \
--wanted_words=$URBAN_WORDS

2019-04-20 02:49:25.684716: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-04-20 02:49:25.684985: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x55904418aec0 executing computations on platform Host. Devices:
2019-04-20 02:49:25.685035: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-04-20 02:49:25.876828: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-04-20 02:49:25.877413: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x55904418b180 executing computations on platform CUDA. Devices:
2019-04-20 02:49:25.877478: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2019-04-20 02:49:25.877853: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1433

Freeze graph to protobuf file

In [55]:
!python ML-KWS-for-MCU/freeze.py \
--clip_duration_ms $TIME_CLIP  \
--window_size_ms 40.0 \
--window_stride_ms 20.0 \
--dct_coefficient_count 10 \
--sample_rate $SAMPLE_RATE \
--model_size_info $MODEL_SIZE  \
--checkpoint work/DS_CNN_UrbanSounds/DS_CNN1/training/best/ds_cnn_6444.ckpt-3600 \
--output_file urban_dscnn.pb \
--wanted_words $URBAN_WORDS 

2019-04-20 03:59:42.593171: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-04-20 03:59:42.593426: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x5601d8e5a940 executing computations on platform Host. Devices:
2019-04-20 03:59:42.593488: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-04-20 03:59:42.790286: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-04-20 03:59:42.790884: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x5601d8e5b700 executing computations on platform CUDA. Devices:
2019-04-20 03:59:42.790918: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2019-04-20 03:59:42.791321: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1433

Test protobuf file
URBAN_WORDS=clock_alarm,sneezing,crying_baby,door_knock,drinking_sipping,dog,coughing,vacuum_cleaner,glass_breaking,laughing

In [56]:
!python ML-KWS-for-MCU/label_wav.py \
--wav /content/Sound-Datasets/ESC50-downsampled_silence_dataset/clock_alarm/3-148297-A.wav \
--graph /content/urban_dscnn.pb \
--labels /content/work/DS_CNN_UrbanSounds/DS_CNN1/training/ds_cnn_labels.txt \
--how_many_labels 10

Instructions for updating:
Use tf.gfile.GFile.
2019-04-20 04:00:17.822175: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-04-20 04:00:17.822399: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x55ede78c1020 executing computations on platform Host. Devices:
2019-04-20 04:00:17.822432: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-04-20 04:00:18.020111: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-04-20 04:00:18.021178: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x55ede78c12e0 executing computations on platform CUDA. Devices:
2019-04-20 04:00:18.021221: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2019-04-20 04:00:18.021642: I tensor

Fold Batchnorm (needed for quantization of ds-cnn)

In [14]:
!python ML-KWS-for-MCU/fold_batchnorm.py \
--summaries_dir work/DS_CNN/DS_CNN2/retrain_logs \
--train_dir work/DS_CNN/DS_CNN2/training \
--data_url= none  \
--data_dir Sound-Datasets/ESC50-downsampled_silence_dataset/ \
--clip_duration_ms $TIME_CLIP  \
--window_size_ms 40.0 \
--window_stride_ms 20.0 \
--dct_coefficient_count 10 \
--sample_rate $SAMPLE_RATE \
--batch_size 100 \
--save_step_interval 200 \
--eval_step_interval 400 \
--testing_percentage 10 \
--validation_percentage 10 \
--unknown_percentage 10.0 \
--silence_percentage 10.0 \
--model_size_info $MODEL_SIZE  \
--checkpoint work/DS_CNN_UrbanSounds/DS_CNN1/training/best/ds_cnn_6444.ckpt-3600 \
--wanted_words $URBAN_WORDS

2019-04-20 02:50:42.968342: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-04-20 02:50:42.968620: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x559f290627e0 executing computations on platform Host. Devices:
2019-04-20 02:50:42.968655: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-04-20 02:50:43.157028: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-04-20 02:50:43.157657: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x559f290635a0 executing computations on platform CUDA. Devices:
2019-04-20 02:50:43.157692: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2019-04-20 02:50:43.158100: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1433

Quantize model to weights.h file

In [53]:
!python ML-KWS-for-MCU/quant_test.py \
--summaries_dir work/DS_CNN/DS_CNN2/retrain_logs \
--train_dir work/DS_CNN/DS_CNN2/training \
--data_url= none  \
--data_dir Sound-Datasets/ESC50-downsampled_silence_dataset/ \
--clip_duration_ms $TIME_CLIP  \
--window_size_ms 40.0 \
--window_stride_ms 20.0 \
--dct_coefficient_count 10 \
--sample_rate $SAMPLE_RATE \
--batch_size 100 \
--save_step_interval 200 \
--eval_step_interval 400 \
--testing_percentage 10 \
--validation_percentage 10 \
--unknown_percentage 10.0 \
--silence_percentage 10.0 \
--model_size_info $MODEL_SIZE   \
--checkpoint work/DS_CNN_UrbanSounds/DS_CNN1/training/best/ds_cnn_6444.ckpt-3600_bnfused \
--wanted_words $URBAN_WORDS \
--act_max 32 32 8 8 8 16 2 32 2 4 2 4 
#--act_max 32 8 32 16 8 64 16 16 16 128 16 32

2019-04-20 03:12:02.965918: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-04-20 03:12:02.966204: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x55cd31c03a20 executing computations on platform Host. Devices:
2019-04-20 03:12:02.966255: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-04-20 03:12:03.151182: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-04-20 03:12:03.151794: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x55cd31c05de0 executing computations on platform CUDA. Devices:
2019-04-20 03:12:03.151827: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2019-04-20 03:12:03.152243: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1433

Download weights.h

In [0]:
from google.colab import files

files.download('weights.h')

Download model as tar file

In [0]:
!tar -cvf dscnn_urban_Dataset_test.tar /content/work/DS_CNN_UrbanSounds/
!ls -l

from google.colab import files

files.download('dscnn_urban_Dataset_test.tar')